# Logical Transduction

Agentics objects are capable of performing logical transduction between their states by using the provied LLMs. The logical transduction operator << can be applied between any two agentics of any type.

Transduction is done between a source and a target AG when connected by the << operator.

The follwing example transduces a Question into an Answer. 

In [1]:
from agentics import Agentics as AG
from pydantic import BaseModel
from typing import Optional
from agentics.core.llm_connections import get_llm_provider


## Define target and source types
class Answer(BaseModel):
    answer: Optional[str] = None
    justification: Optional[str] = None
    confidence: Optional[float] = None


class Question(BaseModel):
    question: Optional[str] = None


## Instantiate the source AG with a question
source = AG(
    atype=Question,
    llm=get_llm_provider(),  ## You can choose between "openai" (i.e. get_llm_provider("openai")), "watsonx", "gemini", "vllm_crewai"
    ## set verbose to true to see the internal agents log. This is optional.
    states=[Question(question="What is the capital of Italy?")],
)

## Instantiate the target AG with a target type. No instances are needed for zero shot transduction
target = AG(
    atype=Answer,  ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"
    verbose_agent=True,
)  ## set verbose to true to see the internal agents log

# Execute logical transduction by using the << operator between source and target AG
answer = await (
    target << source
)  ## Note that << operator is asyncronus and the results should be awaited

# Print the results of the transduction
print(
    answer.pretty_print()
)  ## Note that confidence is a float number, while other fields are strings

2025-09-09 16:50:33.543 | DEBUG    | agentics.core.llm_connections:<module>:119 - AGENTICS is connecting to the following LLM API providers:
2025-09-09 16:50:33.543 | DEBUG    | agentics.core.llm_connections:<module>:122 - 0 - WatsonX
2025-09-09 16:50:33.544 | DEBUG    | agentics.core.llm_connections:<module>:133 - Please add API keys in .env file to add or disconnect providers.
2025-09-09 16:50:33.553 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-09 16:50:33.553 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx']. Using 'watsonx'
2025-09-09 16:50:33.556 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-09 16:50:33.556 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx']. Using 'watsonx'


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7bc7c125-8dbf-428b-a1e1-c2af9b2e3577                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Your task is to transduce a source Pydantic Object into the specified Output type. Generate only slots that    │
│  are logically deduced from the input information, otherwise live then null.                                    │
│                                                                                                                 │
│  Read carefully the following instructions for executing your task:                                             │
│  Generate an object of the specified type from the following input. SOURCE:                                     │
│  {"question": "What is the capital of Italy?"}                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "answer": "Rome",                                                                                            │
│    "justification": "The capital of Italy is a well-known fact, and Rome is the correct answer.",               │
│    "confidence": 1.0                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b45aeb50-90e0-4585-84f5-3ab6bc41e9bb                                                                     │
│  Agent: Task Executor                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7bc7c125-8dbf-428b-a1e1-c2af9b2e3577                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "answer": "Rome",                                                                                            │
│    "justification": "The capital of Italy is a well-known fact, and Rome is the correct answer.",               │
│    "confidence": 1.0                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Atype : <class '__main__.Answer'>
answer: Rome
justification: The capital of Italy is a well-known fact, and Rome is the correct
  answer.
confidence: 1.0




In agentics, lists of strings can be used as sources instead of AG. Those are provided as input for the transduction.

In [2]:
answer = await (AG(atype=Answer) << ["Where is Paris?"])
print(answer.pretty_print())

Atype : <class '__main__.Answer'>
answer: Paris is in France
justification: The question asks for the location of Paris, and based on geographical
  knowledge, Paris is the capital and most populous city of France
confidence: 0.99




## Asyncronous Transduction

When the source AG has more than one state, the << operator perform asyncronous transfuction for each state to the target type. Transductions are executed in batches.

In [3]:
target = AG(
    atype=Answer,
    verbose_transduction=True,  # Set to verbose to see transduction timings and other logs
    transduction_logs_path="/tmp/answers.jsonl",
)  # Optionally write longs of transductions on the specified path
questions = [
    "Where is Paris?",
    "Who is Alberto Sordi",
    "When will climate change be irreverible?",
    "Who is the best Jeopardy player?",
]

answers = await (target << questions)

print(answer.pretty_print())

answers = await (target << questions)

2025-09-09 16:50:38.910 | DEBUG    | agentics.core.agentics:__lshift__:522 - Transduction from input texts ['Where is Paris?', 'Who is Alberto Sordi', 'When will climate change be irreverible?', 'Who is the best Jeopardy player?'] to <class 'pydantic._internal._model_construction.ModelMetaclass'> in progress. This might take a while
2025-09-09 16:50:38.910 | DEBUG    | agentics.core.agentics:__lshift__:582 - transducer class: <class 'agentics.abstractions.pydantic_transducer.TransducerCrewAI'>
2025-09-09 16:50:43.626 | DEBUG    | agentics.core.agentics:__lshift__:604 - Processed 4 states in 4.7152 seconds
2025-09-09 16:50:43.627 | DEBUG    | agentics.core.agentics:__lshift__:522 - Transduction from input texts ['Where is Paris?', 'Who is Alberto Sordi', 'When will climate change be irreverible?', 'Who is the best Jeopardy player?'] to <class 'pydantic._internal._model_construction.ModelMetaclass'> in progress. This might take a while
2025-09-09 16:50:43.628 | DEBUG    | agentics.core.a

Atype : <class '__main__.Answer'>
answer: Paris is in France
justification: The question asks for the location of Paris, and based on geographical
  knowledge, Paris is the capital and most populous city of France
confidence: 0.99




2025-09-09 16:50:51.706 | DEBUG    | agentics.core.agentics:__lshift__:604 - Processed 4 states in 8.0784 seconds


## Self Transduction

Self transduction is a method of AGs that enables async execution of transductions between slots of the same object.

In [4]:
from pydantic import BaseModel, Field
from typing import Optional


## Define the Pydantic type
class Movie(BaseModel):
    movie_name: Optional[str] = (
        None  ## Note that fields name should match the column name in the input csv
    )
    genre: Optional[str] = None
    description: Optional[str] = None
    tweet: Optional[str] = Field(
        None, description="Generate a Tweet to advertise the movie"
    )


movies = AG.from_csv("data/movies.csv", atype=Movie)
movies.verbose_transduction = True
movies.llm = get_llm_provider(
    "watsonx"
)  ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"
movies.states = movies.states[:100]  ## Reduce the number of input movies to 10

movies_with_tweets = await movies.self_transduction(
    ["movie_name", "genre", "description"],  ## source fields
    ["tweet"],  ## target fields
    ## Note that instruction are only needed when the  relation between source and target type
    # is not innediately clear and need to be further specified or disambiguated.
    instructions="Generate a tweet to advertise the release of the input movie",
)

print(movies_with_tweets.pretty_print())

2025-09-09 16:50:51.718 | DEBUG    | agentics.core.agentics:from_csv:289 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-09 16:50:51.727 | DEBUG    | agentics.core.llm_connections:get_llm_provider:42 - Using specified LLM provider: watsonx
2025-09-09 16:50:51.729 | DEBUG    | agentics.core.agentics:__lshift__:495 - Executing task: Generate a tweet to advertise the release of the input movie
100 states will be transduced
2025-09-09 16:50:51.730 | DEBUG    | agentics.core.agentics:__lshift__:582 - transducer class: <class 'agentics.abstractions.pydantic_transducer.TransducerCrewAI'>
2025-09-09 16:51:19.164 | DEBUG    | agentics.core.agentics:__lshift__:604 - Processed 100 states in 27.4329 seconds


Atype : <class '__main__.Movie'>
movie_name: The Shawshank Redemption
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.
tweet: 'Get ready to be inspired! ''The Shawshank Redemption'', a gripping Drama/Crime
  film, is now out! Imprisoned for a crime he didn''t commit, Andy Dufresne''s story
  of hope & redemption will leave you breathless. Don''t miss this classic! #TheShawshankRedemption
  #NewRelease #MustWatch'

movie_name: The Godfather
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patria

As an alternative, self transduction can be encoded using logical transduction algebra which uses the AG() notation to rebind the original data into AGs of the requested subtypes. Learn more about atype manipulation in agentics [here](link)

In [5]:
movies = AG.from_csv("data/movies.csv", atype=Movie)
tweets = await (
    AG(
        atype=movies("tweet").atype,
        instructions="Generate a tweet to advertise the release of the input movie",
    )
    << movies("movie_name", "genre", "description")
)
print(
    tweets.pretty_print()
)  ## Note that differently from self transduction the output tweets
##has only the tweet field, whereas self transduction preserves
## the original source data

2025-09-09 16:51:19.204 | DEBUG    | agentics.core.agentics:from_csv:289 - Importing Agentics of type Movie from CSV data/movies.csv


Atype : <class 'agentics.core.agentics.tweet'>
tweet: 'Get ready to be inspired! ''Forrest Gump'' is now out! This iconic film tells
  the story of a man who defies all odds and achieves greatness, while searching for
  his one true love. Don''t miss out on this unforgettable journey! #ForrestGump #NewRelease
  #MustSeeMovie'

tweet: 'Get ready to be inspired! ''The Shawshank Redemption'', a gripping Drama &
  Crime film, is now out! Imprisoned for a crime he didn''t commit, Andy Dufresne''s
  story of hope & redemption will leave you breathless. Don''t miss it! #TheShawshankRedemption
  #NewRelease #MustWatch'

tweet: 'Get ready for a cinematic masterpiece! ''The Godfather Part II'' is now out!
  This gripping Drama, Crime film follows the Corleone crime family''s saga as they
  expand their business into Las Vegas, Hollywood, and Cuba. Don''t miss it! #TheGodfatherPartII
  #Drama #Crime'

tweet: 'Get ready to witness a powerful true story! ''Schindler''s List'', a gripping
  Drama, H

## Few Shots Transduction

Few shots examples can be provided for transduction by adding instances of the target instances in correspondance to their sources . Those will be used by the LLM to infer by analogy all the Null instances of the target type.


In [6]:
movies = AG.from_csv("data/movies.csv")
movies.states = movies.states[:20]
## Note that obly the first 9 movies have categories
for i, movie in enumerate(movies):
    print(f"{i}: {movie.genre}")
## predicting new genre from given examples
all_genres = await (movies("genre") << movies("movie_name", "description"))
print(all_genres.pretty_print())
## few shots examples can also be used in self transfuction
movies_with_genre = await movies.self_transduction(
    ["movie_name", "description"], ["genre"]
)
print(movies_with_genre.pretty_print())

0: Drama, Crime
1: Drama, Crime
2: Drama, Crime
3: Drama, History, War
4: Drama
5: Animation, Family, Fantasy
6: Drama, Action, Crime, Thriller
7: Comedy, Drama, Romance
8: Fantasy, Drama, Crime
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 
17: 
18: 
19: 
Atype : <class 'agentics.core.agentics.genre'>
genre: Drama, Crime

genre: Drama, History, War

genre: Drama, Crime

genre: Drama, History, War

genre: Drama

genre: Drama, Action, Crime, Thriller

genre: Drama, Action, Crime, Thriller

genre: Comedy, Drama, Romance

genre: Fantasy, Drama, Crime

genre: Drama, Thriller, Comedy

genre: Animation, Fantasy, Romance

genre: Drama, Romance

genre: Fantasy, Adventure, Drama

genre: Drama, Romance

genre: Western, Drama

genre: Action, Adventure, Drama

genre: Drama, Crime

genre: Adventure, Science Fiction

genre: Drama, War

genre: Drama, Comedy


Atype : <class 'agentics.core.utils.AType#movie_name:genre:descriptionOptional'>
movie_name: The Shawshank Redemption
genre: Drama, History, War
descri